## Cleaning

In [1]:
import pandas as pd

base_path = "../datasets"
dataset = pd.read_parquet(f"{base_path}/sample_dataset.parquet", engine="pyarrow")

General

In [2]:
dataset.columns

Index(['summons_number', 'plate_id', 'registration_state', 'plate_type',
       'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make',
       'issuing_agency', 'street_code1', 'street_code2', 'street_code3',
       'vehicle_expiration_date', 'violation_location', 'violation_precinct',
       'issuer_precinct', 'issuer_code', 'issuer_command', 'issuer_squad',
       'violation_time', 'time_first_observed', 'violation_county',
       'violation_in_front_of_or_opposite', 'house_number', 'street_name',
       'intersecting_street', 'date_first_observed', 'law_section',
       'sub_division', 'violation_legal_code', 'days_parking_in_effect',
       'from_hours_in_effect', 'to_hours_in_effect', 'vehicle_color',
       'unregistered_vehicle', 'vehicle_year', 'meter_number',
       'feet_from_curb', 'violation_post_code', 'violation_description',
       'no_standing_or_stopping_violation', 'hydrant_violation',
       'double_parking_violation', 'DataYear'],
      dtype='object')

## Violation

### Dates

First observation datetime

In [3]:
def format_hour(hour_series):
    hour_series = hour_series.str.upper() + "M"
    hour_series = hour_series.str.replace(r'^00', '12', regex=True)
    return pd.to_datetime(hour_series, format="%I%M%p", errors="coerce")

In [4]:
initial_nan = dataset['time_first_observed'].isna().sum()
dataset['time_first_observed'] = format_hour(dataset['time_first_observed'])
final_nan = dataset['time_first_observed'].isna().sum()

print(f"Percentage of initial NaNs: {initial_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of final NaNs: {final_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of NaNs removed: {(final_nan - initial_nan) / len(dataset) * 100:.2f}%")

Percentage of initial NaNs: 93.40%
Percentage of final NaNs: 94.55%
Percentage of NaNs removed: 1.15%


In [5]:
# replace 0 with NaN
dataset['date_first_observed'] = dataset['date_first_observed'].replace('0', pd.NaT)
# replace 0001-01-03T12:00:00.000 with NaN
dataset['date_first_observed'] = dataset['date_first_observed'].replace('0001-01-03T12:00:00.000', pd.NaT)

initial_nan = dataset['date_first_observed'].isna().sum()
dataset['date_first_observed'] = pd.to_datetime(dataset['date_first_observed'], format='%Y%m%d', errors='coerce')
final_nan = dataset['date_first_observed'].isna().sum()

print(f"Percentage of initial NaNs: {initial_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of final NaNs: {final_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of NaNs removed: {(final_nan - initial_nan) / len(dataset) * 100:.2f}%")

Percentage of initial NaNs: 98.76%
Percentage of final NaNs: 98.76%
Percentage of NaNs removed: 0.00%


In [6]:
# merge the date and time
dataset['date_first_observed'] = pd.to_datetime(dataset["date_first_observed"].dt.strftime('%Y-%m-%d') + ' ' + dataset["time_first_observed"].dt.strftime('%H:%M:%S'))
dataset = dataset.drop(["time_first_observed"], axis=1)

Issue Datetime

In [7]:
# convert the date to the same format as the weather data
dataset['issue_date'] = pd.to_datetime(dataset["issue_date"], format="mixed")

# format the hour
dataset['violation_time'] = format_hour(dataset['violation_time'])

# combine the date and time
dataset['issue_date'] = pd.to_datetime(dataset["issue_date"].dt.strftime('%Y-%m-%d') + ' ' + dataset["violation_time"].dt.strftime('%H:%M:%S'))
dataset = dataset.drop(["violation_time"], axis=1)

### Location and codes

In [8]:
# 0 is used as nan in parking violation codes
dataset['violation_code'] = dataset['violation_code'].replace(0, pd.NA)

Violation county

In [9]:
# translate the county names to the borough names
county_to_borough = {
    "BRONX": "BX", # Bronx
    "BX": "BX",
    "Bronx": "BX",
    "BRONX": "BX",
    "BK": "K", # Brooklyn known as Kings
    "K": "K",
    "Kings": "K",
    "KINGS": "K",
    "KING": "K",
    "Q": "Q", # Queens
    "QN": "Q",
    "Qns": "Q",
    "QUEEN": "Q",
    "QUEENS": "Q",
    "QNS": "Q",
    "QU": "Q",
    "NY": "NY", # Manhattan known as New York
    "MN": "NY",
    "MAN": "NY",
    "NEW Y": "NY",
    "NEWY": "NY",
    "NYC": "NY",
    "ST": "R", # Staten Island known as Richmond
    "R": "R",
    "Rich": "R",
    "RICH": "R",
    "RICHM": "R",
    "RC": "R",
    "MH": "NY",
    "MS": "NY",
    "N": "NY",
    "P": "NY",
    "PBX": "NY",
    "USA": "NY",
    "VINIS": "NY",
    "A": pd.NA,
    "F": pd.NA,
    "ABX": pd.NA,
    "108": pd.NA,
    "103": "R", # Staten Island zip code
    "00000": pd.NA,
    "K   F": pd.NA,
}

dataset['violation_county'] = dataset['violation_county'].map(county_to_borough)

Street code

In [10]:
dataset['street_code'] = dataset['street_code1'].where(dataset['street_code1'] != 0, dataset['street_code2'].where(dataset['street_code2'] != 0, dataset['street_code3'])).astype("string")
dataset['street_code'] = dataset['street_code'].replace('0', pd.NA)
dataset = dataset.drop(["street_code1", "street_code2", "street_code3"], axis=1)

violation_location

In [11]:
dataset['violation_location'] = dataset['violation_location'].fillna("0")
dataset['violation_location'] = dataset['violation_location'].astype(int).astype(str)
dataset['violation_location'] = dataset['violation_location'].replace('0', pd.NA)
dataset['violation_location'].value_counts(dropna=False)

violation_location
<NA>    403433
19       53356
14       38131
18       36866
114      33911
         ...  
804          1
164          1
160          1
919          1
57           1
Name: count, Length: 154, dtype: int64

Violation precint is the same as violation location

## Vehicle

### Codes

In [12]:
dataset['vehicle_make'] = dataset['vehicle_make'].str.upper()

In [13]:
# check how many vehicle make values are less than 100
vehicle_make_counts = dataset['vehicle_make'].value_counts()
vehicle_make_counts = vehicle_make_counts[vehicle_make_counts < 100]
print(f"Vehicle make values with less than 100 counts: {len(vehicle_make_counts)}")
print(f'Total vehicle make values: {len(dataset["vehicle_make"].unique())}')
print(f'Percentage of vehicle make values with less than 100 counts: {len(vehicle_make_counts) / len(dataset["vehicle_make"].unique()) * 100:.2f}%')
print(f'Remaining vehicle make values: {len(dataset["vehicle_make"].unique()) - len(vehicle_make_counts)}')

# replace the vehicle make values with less than 100 counts with "OTHER"
dataset['vehicle_make'] = dataset['vehicle_make'].where(~dataset['vehicle_make'].isin(vehicle_make_counts.index), "OTHER")

Vehicle make values with less than 100 counts: 1354
Total vehicle make values: 1447
Percentage of vehicle make values with less than 100 counts: 93.57%
Remaining vehicle make values: 93


### Dates

In [14]:
# replace wrong year with NaN (years greater than now or 0)
current_year = pd.Timestamp.now().year
dataset['vehicle_year'] = dataset['vehicle_year'].where(dataset['vehicle_year'] <= current_year, pd.NA)
dataset['vehicle_year'] = dataset['vehicle_year'].replace(0, pd.NA)

In [15]:
initial_nan = dataset['vehicle_expiration_date'].isna().sum()
dataset['vehicle_expiration_date'] = pd.to_datetime(dataset['vehicle_expiration_date'], format='%Y%m%d', errors='coerce')
final_nan = dataset['vehicle_expiration_date'].isna().sum()

print(f"Percentage of initial NaNs: {initial_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of final NaNs: {final_nan / len(dataset) * 100:.2f}%")
print(f"Percentage of NaNs removed: {(final_nan - initial_nan) / len(dataset) * 100:.2f}%")

Percentage of initial NaNs: 6.85%
Percentage of final NaNs: 60.22%
Percentage of NaNs removed: 53.37%


### Registration

Infos about US state codes and plate types: https://www.nyc.gov/html/dof/html/pdf/faq/stars_codes.pdf

Registration State and Plate Type

In [16]:
codes = "AL ALABAMA MT MONTANA AK ALASKA NE NEBRASKA AZ ARIZONA NV NEVADA AR ARKANSAS NH NEW HAMPSHIRE CA CALIFORNIA NJ NEW JERSEY CO COLORADO NM NEW MEXICO CT CONNECTICUT NY NEW YORK DE DELAWARE NC NORTH CAROLINA FL FLORIDA ND NORTH DAKOTA GA GEORGIA OH OHIO HI HAWAII OK OKLAHOMA ID IDAHO OR OREGON IL ILLINOIS PA PENNSYLVANIA IN INDIANA RI RHODE ISLAND IA IOWA SC SOUTH CAROLINA KS KANSAS SD SOUTH DAKOTA KY KENTUCKY TN TENNESSEE LA LOUISIANA TX TEXAS ME MAINE UT UTAH MD MARYLAND VT VERMONT MA MASSACHUSETTS VA VIRGINIA MI MICHIGAN WA WASHINGTON MN MINNESOTA WV WEST VIRGINIA MS MISSISSIPPI WI WISCONSIN MO MISSOURI WY WYOMING AB Alberta ON Ontario BC British Columbia PE Prince Edward Island FO Foreign QB Quebec MB Manitoba SK Saskatchewan MX Mexico GV U.S. Government NB New Brunswick DP U.S. State Dept. NF Newfoundland DC Washington D.C. NT Northwest Territories YT Yukon Territory NS Nova Scotia"
codes = codes.split()
codes = [code for code in codes if code.isupper() and len(code) == 2]
print(f'Total number of codes: {len(codes)}')

plate_ids = "AGR Agricultural Vehicle MCD Motorcycle Dealer AMB Ambulance MCL Marine Corps League ARG Air National Guard MED Medical Doctor ATD All Terrain Deale MOT Motorcycle ATV All Terrain Vehicle NLM Naval Militia AYG Army National Guard NYA New York Assembly BOB Birthplace of Baseball NYC New York City Council BOT Boat NYS New York Senate CBS County Bd. of Supervisors OMF Omnibus Public Service CCK County Clerk OML Livery CHC  Household Carrier (Com) OMO Omnibus Out-of-State CLG County Legislators OMR Bus CMB Combination - Connecticut OMS Rental CME  Coroner Medical Examiner OMT Taxi CMH Congress. Medal of Honor OMV Omnibus Vanity COM Commercial Vehicle ORC Organization (Com) CSP Sports (Com) ORG Organization (Pas) DLR Dealer PAS Passenger EDU Educator PHS Pearl Harbor Survivors FAR Farm vehicle PPH Purple Heart FPW Former Prisoner of War PSD Political Subd. (Official) GAC Governor's Additional Car RGC Regional (Com) GFC Gift Certificate RGL Regional (Pas) GSC Governor's Second Car SCL School Car GSM Gold Star Mothers SNO Snowmobile HAC Ham Operator Comm SOS Survivors of the Shield HAM Ham Operator SPC Special Purpose Comm. HIF Special Reg.Hearse SPO Sports HIR Hearse Coach SRF Special Passenger - Vanity HIS Historical Vehicle SRN Special Passenger - Judges HOU House/Coach Trailer STA State Agencies HSM Historical Motorcycle STG State National Guard IRP Intl. Registration Plan SUP Justice Supreme Court ITP In Transit Permit TOW Tow Truck JCA Justice Court of Appeals TRA Transporter JCL Justice Court of Claims THC Household Carrier Tractor JSC Supreme Court Appl. Div TRC Tractor Regular JWV Jewish War Veterans TRL Trailer Regular LMA Class A Limited Use Mtrcyl. USC U. S. Congress LMB Class B Limited Use Mtrcyl. USS U. S. Senate LMC Class C Limited Use Mtrcyl. VAS Voluntary Ambulance Svc. LOC Locomotive VPL Van Pool LTR Light Trailer WUG World University Games LUA Limited Use Automobile"
plate_ids = plate_ids.split()
plate_ids = [plate_id for plate_id in plate_ids if plate_id.isupper() and len(plate_id) == 3]
print(f'Total number of plate ids: {len(plate_ids)}')

Total number of codes: 67
Total number of plate ids: 87


In [17]:
# Registration state should be in the codes
dataset['registration_state'] = dataset['registration_state'].where(dataset['registration_state'].isin(codes), pd.NA)

# Plate type should be in the plate ids
dataset['plate_type'] = dataset['plate_type'].where(dataset['plate_type'].isin(plate_ids), pd.NA)

Plate ID

In [18]:
dataset['plate_id'] = dataset['plate_id'].str.upper()
dataset['plate_id'] = dataset['plate_id'].replace('BLANKPLATE', pd.NA)

# if the plate_id is less than 5 characters, replace it with NaN
dataset['plate_id'] = dataset['plate_id'].where(dataset['plate_id'].str.len() >= 5, pd.NA)

# if the plate_id is more than 8 characters, replace it with NaN
dataset['plate_id'] = dataset['plate_id'].where(dataset['plate_id'].str.len() <= 8, pd.NA)

# if the plate_id contains special characters, replace it with NaN
dataset['plate_id'] = dataset['plate_id'].where(dataset['plate_id'].str.match(r'^[A-Z0-9]*$'), pd.NA)

vehicle body type: Too many vehicle body types and too many spellings to clean

## Precint

issuing_agency: Not really understandable

violation_precinct

In [19]:
dataset['violation_precinct'] = dataset['violation_precinct'].replace(0, pd.NA)

differences_precint_violation = dataset['violation_location'].astype(str) != dataset['violation_precinct'].astype(str)
differences_precint_violation = differences_precint_violation.sum()
print(f"Differences between precinct and violation location: {differences_precint_violation}")

Differences between precinct and violation location: 0


In [20]:
dataset = dataset.drop(["violation_location"], axis=1)

Issuer_precinct, issuer_code, issuer_command, issuer_squad

In [21]:
dataset['issuer_precinct'] = dataset['issuer_precinct'].replace(0, pd.NA)
dataset['issuer_code'] = dataset['issuer_code'].replace(0, pd.NA)

- Law_section: Section of the law or code related to the violation
- sub_division: Sub-division or additional information related to the violation
- violation_legal_code: Legal code related to the violation (0 or T)
- days_parking_in_effect: Days of the week when parking regulations are in effect (insert B as blank space and make sure every item has 7 chars)

In [22]:
dataset['law_section'] = dataset['law_section'].replace(0, pd.NA)

# Replace missing values (NA) with an empty string
dataset['days_parking_in_effect'] = dataset['days_parking_in_effect'].fillna('')

# Define a function to pad and replace blank spots with 'B'
def pad_parking_days(parking_days):
    # Replace spaces with 'B' and pad the string to 7 characters with 'B'
    parking_days = parking_days.replace(' ', 'B')
    return parking_days.ljust(7, 'B')

dataset['days_parking_in_effect'] = dataset['days_parking_in_effect'].apply(pad_parking_days)

- from_hours_in_effect: Starting time when parking regulations are in effect
- to_hours_in_effect: Ending time when parking regulations are in effect

In [23]:
dataset['from_hours_in_effect'] = format_hour(dataset['from_hours_in_effect']).dt.time
dataset['to_hours_in_effect'] = format_hour(dataset['to_hours_in_effect']).dt.time

- vehicle_color: too many colors to clean
- unregistered_vehicle: change to True/False
- meter_number: Number
- feet_from_curb: No issues
- violation_post_code: Not sure why it has letters on it
- violation_description: Textual details
- no_standing_or_stopping_violation, hydrant_violation and double_parking_violation: seems to be all None

In [25]:
dataset['unregistered_vehicle'] = dataset['unregistered_vehicle'].notna() & (dataset['unregistered_vehicle'] == '0')
dataset['unregistered_vehicle'] = dataset['unregistered_vehicle'].astype(bool)

In [33]:
dataset['meter_number'] = dataset['meter_number'].replace('-', pd.NA)

double_parking_violation
<NA>    1404261
Name: count, dtype: Int64

### Final result

In [29]:
dataset.head()

,summons_number,plate_id,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,...,vehicle_year,meter_number,feet_from_curb,violation_post_code,violation_description,no_standing_or_stopping_violation,hydrant_violation,double_parking_violation,DataYear,street_code
30356,5069663409,HSP9388,PA,PAS,2013-07-20 18:09:00,7,CP,DODGE,V,NaT,...,1998,<NA>,0,<NA>,FAILURE TO STOP AT RED LIGHT,<NA>,<NA>,<NA>,2014,<NA>
87255,7433097510,T466979C,NY,OMT,2013-07-29 08:06:00,21,SUBN,TOYOT,T,2014-06-30,...,2013,<NA>,0,H -,21-No Parking (street clean),<NA>,<NA>,<NA>,2014,51090
11675,1358115771,49722JG,NY,COM,2013-07-10 09:44:00,19,VAN,MAZDA,X,NaT,...,1995,-,0,<NA>,<NA>,<NA>,<NA>,<NA>,2014,<NA>
91091,7780960298,K5669F,TN,PAS,2013-07-29 14:36:00,14,4DSD,DODGE,T,NaT,...,<NA>,<NA>,0,35 -,14-No Standing,<NA>,<NA>,<NA>,2014,68020
36443,5069699234,EEV3130,NY,PAS,2013-07-21 22:07:00,7,2DSD,ME/BE,V,NaT,...,2004,<NA>,0,<NA>,FAILURE TO STOP AT RED LIGHT,<NA>,<NA>,<NA>,2014,<NA>


In [10]:
dataset.to_parquet(f"{base_path}/sample_data_cleaned.parquet", engine="pyarrow")